In [1]:
%%html
<style type="text/css">
  span.ecb { background: yellow; }
</style>

<span class="ecb">Comments by ECB</span>

# ATMO 5331 - Homework 2 - Fall 2023
## Due 24 Sep, 2023 (Sunday, 11:59 pm)

When doing this homework, remember that you have three jobs:
1. Make it work and get the right answer.
2. Clean it up so that I can understand what you've done. If you think I might not undersand, document it with a comment or a function docstring.
3. Practice _generalizing_ your thinking: write code that is tolerant of changes to the specifics of a problem, but not the structure of the problem.

You should present your work with a clear logical progression. If that seems like a hassle, remember that in doing so you are practicing skills that are expected in your thesis and journal publications.

You may work alone or in pairs. I will not be adjudicating relative level of effort in group work, so you are responsible for ensuring there is an even contribution by your partner.

**Question 1**

Grab the [WGS84 implementation manual](https://www.icao.int/safety/pbn/documentation/eurocontrol/eurocontrol%20wgs%2084%20implementation%20manual.pdf), and implement a translation from geodetic latitude, longitude, and altitude (referenced to the WGS84 ellipsoid) to the local XYZ cartesian system used for WGS84.

Use part 1 of Helmert's formula on p. 81 (Appendix E), and refer to Fig. B-6 in Appendix B (p. 70) for information about the coordinate system notation.

Careful with degrees and radians.

Compare your results to what you get when using the `proj4` library. This library has its origins in public domain code written by the USGS, and is used in many open source packages, including the QGIS system. For easy use of the `proj4` library, we will use the helper routines in `coords.py`. I use these same helpers all the time in practice, and this code is running in operations in NOAA.

You set up a coordinate system transform object as shown below. It defaults to a WGS84 ellipsoid, so we don't have to specify that. Once the coordinate system object `geo` has been created, you can reuse it withouth calling `GeographicSystem()` again. It accepts arrays of data.
```
from coords import GeographicSystem 
geo = GeographicSystem() 
X, Y, Z = geo.toECEF(lon, lat, alt) # Use degrees
```

For your dataset, please use: 
```
import numpy as np
lat = np.array([  33.5,   1.0,   0.0,   0.0,   0.0,  10.0, -10.0]) 
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0]) 
alt = np.zeros_like(lat)
```

Demonstrate that your ECEF conversion equals that provided by the coordinate system library.


### SOLUTION
   - This will be provided in 3-parts: Helmert, Coords and comparism of both results

### (a.) Program implementing the `Helmerts_formula` for the `geodetic-to-ECEF conversion`

In [10]:
import numpy as np

def helmerts_formula(lat, lon, alt, a, e):
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)

    # Calculate the radius of curvature in the prime vertical (N)
    N = a / np.sqrt(1 - e**2 * np.sin(lat_rad)**2)

    # Calculate X, Y, and Z using Helmert's Formula
    X = (N + alt) * np.cos(lat_rad) * np.cos(lon_rad)
    Y = (N + alt) * np.cos(lat_rad) * np.sin(lon_rad)
    Z = ((1 - e**2) * N + alt) * np.sin(lat_rad)

    return X, Y, Z

# Define our dataset as provided by Dr. Eric
lat = np.array([33.5, 1.0, 0.0, 0.0, 0.0, 10.0, -10.0])
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0])
alt = np.zeros_like(lat)  # Assuming altitudes are all zero for this example

# Defining the constants for the WGS84 ellipsoid
a = 6378137.0  # Semi-major axis (meters)
e = 0.0818191908426  # Eccentricity

# Here, we perform the geodetic-to-ECEF conversion 
X, Y, Z = helmerts_formula(lat, lon, alt, a, e)

# Format and display the results
for i in range(len(lat)):
    print(f"Point {i + 1}: X={X[i]}, Y={Y[i]}, Z={Z[i]}")


Point 1: X=-1061448.7541803515, Y=-5217187.307231329, Z=3500334.288022366
Point 2: X=1650533.5883109367, Y=-6159875.211175389, Z=110568.77482456704
Point 3: X=555891.26758132, Y=-6353866.263102791, Z=0.0
Point 4: X=2695517.1720840395, Y=-5780555.229886577, Z=0.0
Point 5: X=1650783.3278730563, Y=-6160807.251909879, Z=0.0
Point 6: X=1625868.3272134357, Y=-6067823.203577563, Z=1100248.5477353656
Point 7: X=1625868.3272134357, Y=-6067823.203577563, Z=-1100248.5477353656


### (b.) Program implementing `coods.py` for the `geodetic-to-ECEF conversion`
- `The coords.py module defines a set of classes and functions for handling various coordinate systems and transformations, including geographic systems, map projections, radar coordinate systems, and tangent plane Cartesian systems. These classes and functions make use of the proj4 library to perform coordinate transformations.` (*Attached only for my consumption and future reference*)

In [11]:
import numpy as np
from coords import GeographicSystem

# Create a GeographicSystem object (defaults to WGS84 ellipsoid)
geo = GeographicSystem()

# Define our dataset as provided by Dr. Eric
lat = np.array([33.5, 1.0, 0.0, 0.0, 0.0, 10.0, -10.0])
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0])
alt = np.zeros_like(lat)  # Assuming altitudes are all zero for this example

# Perform the geodetic-to-ECEF conversion using the GeographicSystem object
X, Y, Z = geo.toECEF(lon, lat, alt)

# Format and display the results
for i in range(len(lat)):
    print(f"Point {i + 1}: X={X[i]}, Y={Y[i]}, Z={Z[i]}")


Point 1: X=-1061448.7541803522, Y=-5217187.307231333, Z=3500334.2880223556
Point 2: X=1650533.5883109367, Y=-6159875.211175389, Z=110568.77482456664
Point 3: X=555891.26758132, Y=-6353866.263102791, Z=0.0
Point 4: X=2695517.1720840395, Y=-5780555.229886577, Z=0.0
Point 5: X=1650783.3278730563, Y=-6160807.251909879, Z=0.0
Point 6: X=1625868.3272134357, Y=-6067823.203577563, Z=1100248.5477353614
Point 7: X=1625868.3272134357, Y=-6067823.203577563, Z=-1100248.5477353614


### (c.) Program to compare the result of `Helmert's Formula` with that of `Coords.py`

In [12]:
import numpy as np

def helmerts_formula(lat, lon, alt, a, e):
    # Convert latitude and longitude from degrees to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)

    # Calculate the radius of curvature in the prime vertical (N)
    N = a / np.sqrt(1 - e**2 * np.sin(lat_rad)**2)

    # Calculate X, Y, and Z using Helmert's Formula
    X = (N + alt) * np.cos(lat_rad) * np.cos(lon_rad)
    Y = (N + alt) * np.cos(lat_rad) * np.sin(lon_rad)
    Z = ((1 - e**2) * N + alt) * np.sin(lat_rad)

    return X, Y, Z

# Example usage:
lat = np.array([33.5, 1.0, 0.0, 0.0, 0.0, 10.0, -10.0])
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0])
alt = np.zeros_like(lat)  # Assuming altitudes are all zero for this example

# Constants for the WGS84 ellipsoid
a = 6378137.0  # Semi-major axis (meters)
e = 0.0818191908426  # Eccentricity

# Perform the conversion using Helmert's Formula
X_helmert, Y_helmert, Z_helmert = helmerts_formula(lat, lon, alt, a, e)

# Import and use the coord_sys_library (GeographicSystem)
from coords import GeographicSystem
geo = GeographicSystem()
X_geo, Y_geo, Z_geo = geo.toECEF(lon, lat, alt)

# Compare the result of ECEF conversion (Helmert) with coordinate system library (Coords)
for i in range(len(lat)):
    print(f"Point {i + 1}:")
    print(f"Helmert_Formula - X={X_helmert[i]}, Y={Y_helmert[i]}, Z={Z_helmert[i]}")
    print(f"coord_Sys_Library - X={X_geo[i]}, Y={Y_geo[i]}, Z={Z_geo[i]}")
    print()  # Blank line for separation


Point 1:
Helmert_Formula - X=-1061448.7541803515, Y=-5217187.307231329, Z=3500334.288022366
coord_Sys_Library - X=-1061448.7541803522, Y=-5217187.307231333, Z=3500334.2880223556

Point 2:
Helmert_Formula - X=1650533.5883109367, Y=-6159875.211175389, Z=110568.77482456704
coord_Sys_Library - X=1650533.5883109367, Y=-6159875.211175389, Z=110568.77482456664

Point 3:
Helmert_Formula - X=555891.26758132, Y=-6353866.263102791, Z=0.0
coord_Sys_Library - X=555891.26758132, Y=-6353866.263102791, Z=0.0

Point 4:
Helmert_Formula - X=2695517.1720840395, Y=-5780555.229886577, Z=0.0
coord_Sys_Library - X=2695517.1720840395, Y=-5780555.229886577, Z=0.0

Point 5:
Helmert_Formula - X=1650783.3278730563, Y=-6160807.251909879, Z=0.0
coord_Sys_Library - X=1650783.3278730563, Y=-6160807.251909879, Z=0.0

Point 6:
Helmert_Formula - X=1625868.3272134357, Y=-6067823.203577563, Z=1100248.5477353656
coord_Sys_Library - X=1625868.3272134357, Y=-6067823.203577563, Z=1100248.5477353614

Point 7:
Helmert_Formula - 

- NOTE: *Both `HELMERT` and `COORDS` produced the same result*

**Question 2.**

Using the `TangentPlaneCartesianSystem` class, convert the geodetic coordinates to local $(x, y, z)$. Create three tangent planes:

- A tangent plane centered at the MCOM building on the TTU campus, at the height of the ground at that location.

- A tangent plane centered at the MCOM building on the TTU campus, at the ellipsoid.

- A tangent plane directly below the GOES-East satellite at -75.0 degrees longitude.

Use `TangentPlaneCartesianSystem?` in the notebook to learn about the arguments accepted by the projection class. It has the same `.toECEF` and `.fromECEF` methods as the `GeographicSystem`.

Transform the geodetic dataset from the first problem into coordiantes with respect to each tangent plane.

You do not need to use the NAD83 locations of MCOM. They are only there to show my work on how I obtained the vertical position of MCOM in WGS84.

**a.** Using only the GOES-East tangent plane and the transformed geodetic dataset, show that the WGS84 earth shape is not spherical.

**b.** What is a rough, easily memorable rule of thumb for the number of kilometers per degree latitude?

**c.** Print out the tangent plane $(x,y,z)$ of the zeroth data point (it is a bit east of Lubbock). Explain why the differences in the coordinates of the two MCOM tangent planes make sense.

**d.** Imagine that there was no terrain, so that a radar located at MCOM was precisely on the WGS84 ellipsoid. If that radar were to scan toward the zeroth position at 0° elevation angle (assume no atmospheric refraction), how high above the ground would the beam be? Is this disance measured perpendicular to the ellipsoid or perpendicular to the tangent plane?

**e.** Transform your coordiantes back to ECEF from each tangent plane and show they're equal.

In [ ]:
from coords import TangentPlaneCartesianSystem

# From USGS Elevation point query service
# https://nationalmap.gov/epqs/
# NAD83 lon, lat and NAVD88 vertical
mcom_lat_nad83, mcom_lon_nad83 = 33.581857, -101.880360 # NAD83
mcom_alt_nad83 = 983.15

# Using https://vdatum.noaa.gov/vdatumweb/, convert the above to "WGS84 G1674 (Use ITRF2008)"
mcom_lon, mcom_lat = -101.8803718553, 33.5818617015
mcom_alt = 957.179

# The altitude difference is about the height of the geoid at this location.

#### SOLUTION

#### Program to create the tangent planes for the three scenarios and performing the necessary calculations:

In [22]:
import numpy as np
from coords import TangentPlaneCartesianSystem

# Define the geodetic coordinates of the MCOM building on the TTU campus
mcom_lat_wgs84 = 33.5818617015
mcom_lon_wgs84 = -101.8803718553
mcom_alt_wgs84 = 957.179

# Define the geodetic coordinates of the GOES-East satellite location
goes_east_lat_wgs84 = 0.0
goes_east_lon_wgs84 = -75.0
goes_east_alt_wgs84 = 0.0  # Assuming the satellite is at sea level based on given data

# Define the provided geodetic dataset
lat = np.array([  33.5,   1.0,   0.0,   0.0,   0.0,  10.0, -10.0])
lon = np.array([-101.5, -75.0, -85.0, -65.0, -75.0, -75.0, -75.0])
alt = np.zeros_like(lat)

# Create TangentPlaneCartesianSystem instances for the three tangent planes
tangent_plane_mcom_ground = TangentPlaneCartesianSystem(
    mcom_lon_wgs84, mcom_lat_wgs84, mcom_alt_wgs84
)

tangent_plane_mcom_ellipsoid = TangentPlaneCartesianSystem(
    mcom_lon_wgs84, mcom_lat_wgs84, 0.0  # At the ellipsoid
)

tangent_plane_goes_east = TangentPlaneCartesianSystem(
    goes_east_lon_wgs84, goes_east_lat_wgs84, goes_east_alt_wgs84
)

# Transform the geodetic dataset to the three tangent planes
X_mcom_ground, Y_mcom_ground, Z_mcom_ground = tangent_plane_mcom_ground.toECEF(lon, lat, alt)
X_mcom_ellipsoid, Y_mcom_ellipsoid, Z_mcom_ellipsoid = tangent_plane_mcom_ellipsoid.toECEF(lon, lat, alt)
X_goes_east, Y_goes_east, Z_goes_east = tangent_plane_goes_east.toECEF(lon, lat, alt)

# Now we have the Cartesian coordinates in the three tangent planes
# X_mcom_ground, Y_mcom_ground, Z_mcom_ground: MCOM building's ground
# X_mcom_ellipsoid, Y_mcom_ellipsoid, Z_mcom_ellipsoid: MCOM building's ellipsoid
# X_goes_east, Y_goes_east, Z_goes_east: GOES-East satellite location


C:\Users\dayo\Desktop\data-fields\coords.py:390: RuntimeWarning: invalid value encountered in subtract
  n = aboveCenterECEF - self.centerECEF
C:\Users\dayo\Desktop\data-fields\coords.py:390: RuntimeWarning: invalid value encountered in subtract
  n = aboveCenterECEF - self.centerECEF


### Answer to Part a
- (a.) Calculate the differences between the GOES-East tangent plane and the transformed geodetic dataset

In [18]:
# Differences in X, Y, and Z coordinates between the geodetic dataset
# transformed to the GOES-East tangent plane and the actual geodetic coordinates.

delta_X = X - X_goes_east  # Difference in X coordinates
delta_Y = Y - Y_goes_east  # Difference in Y coordinates
delta_Z = Z - Z_goes_east  # Difference in Z coordinates

# Output the delta values
for i in range(len(lat)):
    print(f"Point {i + 1}: Delta_X={delta_X[i]}, Delta_Y={delta_Y[i]}, Delta_Z={delta_Z[i]}")


Point 1: Delta_X=-2717444.0650600377, Delta_Y=-5217085.807231333, Delta_Z=9639091.299942587
Point 2: Delta_X=-5430.32997939433, Delta_Y=-6159800.211175389, Delta_Z=6249334.198363762
Point 3: Delta_X=-1100071.684783185, Delta_Y=-6353781.263102791, Delta_Z=6138765.682358242
Point 4: Delta_X=1039554.2197195347, Delta_Y=-5780490.229886577, Delta_Z=6138765.682358242
Point 5: Delta_X=-5179.624491448514, Delta_Y=-6160732.251909879, Delta_Z=6138765.682358242
Point 6: Delta_X=-30104.28440933209, Delta_Y=-6067748.203577563, Delta_Z=7239011.641903152
Point 7: Delta_X=-30084.965892806184, Delta_Y=-6067748.203577563, Delta_Z=5038519.7228133315


### `Analysis of part a (Coordinate Differences)`

### `Point 1`
- **Delta_X:** -2,717,444.07 meters
- **Delta_Y:** -5,217,085.81 meters
- **Delta_Z:** 9,639,091.30 meters
- Explanation: At Point 1, the coordinates in the GOES-East tangent plane significantly differ from the geodetic coordinates. Specifically, the X and Y coordinates are shifted by millions of meters, and the Z coordinate (altitude) is shifted significantly, highlighting the non-spherical nature of the Earth.

### `The differences are also large for Points 2, 3, 4, 5, 6, & 7`

These results clearly demonstrate that the Earth's shape is not spherical, as the differences between the geodetic coordinates and the tangent plane coordinates are substantial. This is expected because the Earth is an oblate spheroid, meaning it is flattened at the poles and bulging at the equator. 
- `The deviations from a sphere are expected to be more pronounced at higher latitudes`.


### Answer to Part b
- Rule of thumb, it is Approximately 111.32 kilometers per degree latitude.

 - -`This value represents the length of one degree of latitude on the Earth's surface.`

### Answer to Part c

- **c.** Print out the tangent plane $(x,y,z)$ of the zeroth data point (it is a bit east of Lubbock). Explain why the differences in the coordinates of the two MCOM tangent planes make sense.

In [24]:
# The tangent plane (𝑥,𝑦,𝑧) coordinates of the zeroth data point

# Extract the coordinates of the zeroth data point from the transformed MCOM ground tangent plane
X_mcom_ground_zeroth = X_mcom_ground[0]
Y_mcom_ground_zeroth = Y_mcom_ground[0]
Z_mcom_ground_zeroth = Z_mcom_ground[0]

# Extract the coordinates of the zeroth data point from the transformed MCOM ellipsoid tangent plane
X_mcom_ellipsoid_zeroth = X_mcom_ellipsoid[0]
Y_mcom_ellipsoid_zeroth = Y_mcom_ellipsoid[0]
Z_mcom_ellipsoid_zeroth = Z_mcom_ellipsoid[0]

# Print the coordinates
print("Tangent Plane (MCOM Ground) Coordinates for Zeroth Data Point:")
print(f"X={X_mcom_ground_zeroth}, Y={Y_mcom_ground_zeroth}, Z={Z_mcom_ground_zeroth}")

print("\nTangent Plane (MCOM Ellipsoid) Coordinates for Zeroth Data Point:")
print(f"X={X_mcom_ellipsoid_zeroth}, Y={Y_mcom_ellipsoid_zeroth}, Z={Z_mcom_ellipsoid_zeroth}")

# Explain the differences between the two MCOM tangent planes

# Calculate the differences in coordinates between MCOM Ground and MCOM Ellipsoid tangent planes
delta_X_mcom = X_mcom_ground_zeroth - X_mcom_ellipsoid_zeroth
delta_Y_mcom = Y_mcom_ground_zeroth - Y_mcom_ellipsoid_zeroth
delta_Z_mcom = Z_mcom_ground_zeroth - Z_mcom_ellipsoid_zeroth



Tangent Plane (MCOM Ground) Coordinates for Zeroth Data Point:
X=nan, Y=nan, Z=nan

Tangent Plane (MCOM Ellipsoid) Coordinates for Zeroth Data Point:
X=nan, Y=nan, Z=nan


- #### I know the answers here can not be `nan`. So obvioulsy, something is wrong.

**Question 3** Use the `GeostationaryFixedGridSystem` to define two more coordiante transformations for the GOES-East and GOES-West locations at -75.0 and -135.0 degrees longitude.

Use `GeostationaryFixedGridSystem?` in the notebook to learn about the arguments accepted by the projection class.

Convert the dataset to fixed grid coordinates.

For more on fixed grid coordiantes, you can read the [GOES-R L1b product users' guide](https://www.goes-r.gov/resources/docs.html) and [my own description of GOES fixed grid coordiantes](https://github.com/deeplycloudy/glmtools/blob/master/docs/fixedgridguide.md), a.k.a. [the geostationary projection](https://proj.org/operations/projections/geos.html).



In [ ]:
from coords import GeostationaryFixedGridSystem


**Question 4.** Make a plot of the east and north coordinate data from the lon/lat, 3 tangent plane, and two GOES fixed grid projections, for a total of 6 plots.

Transform and plot three more locations (your hometown, your undergraduate institution's location, and the farthest you've been from home).

Does everything make sense? If not, what do you observe? Offer a plausible explanation for what might have happened.

Label the coordinates with the altitude of the point in that coordinate reference system.

In [ ]:
loc_lon = 
loc_lat = 
loc_alt = 


%matplotlib notebook
import matplotlib.pyplot as plt
n_rows, n_cols = 3, 2
fig, axes = plt.subplots(n_rows, n_cols, figsize=(8,8))

axes[0,0].plot
axes[0,0].set_xlabel('Longitude')
axes[0,0].set_ylabel('Latitude')
axes[0,0].set_title('WGS84')
axes[0,0].plot(lon, lat, marker='.', linestyle='none')
axes[0,0].plot(loc_lon, loc_lat, marker='.', linestyle='none')
for tlon, tlat, tlabel in zip(lon, lat, alt):
    axes[0,0].text(tlon, tlat, tlabel)
for tlon, tlat, tlabel in zip(loc_lon, loc_lat, loc_alt):
    axes[0,0].text(tlon, tlat, tlabel)


# Make the other five panels here
    
    
fig.tight_layout()

In [ ]:
# BONUS! Make a 3D plot of all locations. 
# Try to imagine the curved earth surface on which they reside.
# This part is not graded, but might be useful to you.

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')

ax.plot(X, Y, Z, marker='o', linestyle='none') # original locations
ax.plot(locX, locY, locZ, marker='o', linestyle='none') # personal locations
ax.set_xlabel('ECEF X (m)')
ax.set_ylabel('ECEF Y (m)')
ax.set_zlabel('ECEF Z (m)')
# ax.set_aspect('equal')

**5.** Using the arrays you created in the previous assignment, create a `pcolormesh` plot of the data in geostationary coordinates from both the GOES East and GOES West positions. (15 pts.)

**6.** One thing we didn't do in the previous assignment was plot in a "traditional" map projection. We'll do that now with the Azimuthal Equidistant and Gnomonic projections, centered on MCOM, as defined below. The MapProjection class has the same to/from ECEF methods, and coordinates returned are in meters relative to the center point. (15 pts.)

If you're curious, you can peruse [the full list of projections](https://proj.org/operations/projections/index.html) to see how to define others.

Create a plot of the same data in each map projection, and set the axis limits to +/- 1600 km. Do you notice any differences in the two projections?

In [ ]:
aeqd = MapProjection(projection='aeqd', lon_0=mcom_lon, lat_0=mcom_lat)
gnom = MapProjection(projection='gnom', lon_0=mcom_lon, lat_0=mcom_lat)

